# MIMIC Notes Pre-Processing

Pre-processing MIMIC notes for language model and CUIs

Below is a list of redacted items with an example and the replacement token.

Redacted items:
* [x] First Name: `[**First Name (Titles) 137**]`, `t_firstname`
* [x] Last Name: `[**Last Name (Titles) **]`, `t_lastname`
* [x] Initials: `[**Initials (NamePattern4) **]`, `t_initials`
* [x] Name: `[**Name (NI) **]`, `t_name`
* [x] Doctor First Name: `[**Doctor First Name 1266**]`, `t_doctor_firstname`
* [x] Doctor Last Name: `[**Doctor Last Name 1266**]`, `t_doctor_lastname`
* [x] Known Last Name: `[**Known lastname 658**]`, `t_lastname`
* [x] Hospital: `[**Hospital1 **]`, `t_hospital`
* [x] Hospital Unit Name: `**Hospital Unit Name 10**`, `t_hospital`
* [x] Company: `[**Company 12924**]`, `t_workplace`
* [x] University/College: `[**University/College **]`, `t_workplace`
* [x] Date of format YYYY-M-DD: `[**2112-4-18**]`, `t_fulldate`
* [x] Year: `[**Year (4 digits) **]`, `t_year`
* [x] Year YYYY format: `[**2119**]`, `t_year` - I use a regex `\b\d{4}\b` that will match **any** 4 digits which might be problematic, but for the most part 4 digits by itself seems to indicate a year.
* [x] Date of format M-DD: `[**6-12**]`, `[**12/2151**]`, `t_monthday`
* [x] Month/Day: `[**Month/Day (2) 509**]`, `t_monthday`
* [x] Month (only): `[**Month (only) 51**]`, `t_month`
* [x] Holiday: `[**Holiday 3470**]`, `t_month`
* [x] Date Range: `[**Date range (1) 7610**]`, `t_daterange`
* [x] Country: `[**Country 9958**]`, `t_country`
* [x] State: `[**State 3283**]`, `t_state`
* [x] Location: `**Location (un) 2432**`, `t_location`
* [x] Telephone/Fax: `[**Telephone/Fax (3) 8049**]`, `t_phone`
* [x] Clip Number: `[**Clip Number (Radiology) 29923**]`, `t_radclip_id`
* [x] Pager Numeric Identifier: `[**Numeric Identifier 6403**]`, `t_pager_id`
* [x] Pager Number: `[**Pager number 13866**]`, `t_pager_id`
* [x] Social Security Number: `[**Security Number 10198**]`, `t_ssn`
* [x] Serial Number: `[**Serial Number 3567**]`, `t_sn`
* [x] Medical Record Number: `[**Medical Record Number **]`, `t_mrn`
* [x] Provider Number: `[**Provider Number 12521**]`, `t_provider_no`
* [x] Age over 90: `[**Age over 90 **]`, `t_oldage`
* [x] Time: `12:52 PM`, split into 6 segments by the hour and replace with the following tokens: `midnight, dawn, forenoon, afternoon, dusk, night`
* Just numbers: `[** 7901**]`
* Wardname
* Pharmacy MD Number* 

## Imports and Inits

In [1]:
import pandas as pd
import psycopg2
import numpy as np
import re
import random
import datetime

In [2]:
from process_notes import *

## Grab sample data from MIMIC

In [3]:
cats = pd.read_csv('cats.csv')
max_limit = 100

queries = []
for category, n_notes in zip(cats['category'], cats['number_of_notes']):
    limit = min(max_limit, n_notes) if max_limit > 0 else n_notes
    if limit == max_limit:
        q = f"""
        select category, text from correctnotes where category=\'{category}\' order by random() limit {limit};
        """
    else:
        q = f"""
        select category, text from correctnotes where category=\'{category}\';
        """
    queries.append(q)

In [4]:
# limit = 50
# queries = [
#     f"""
#     select category, text from correctnotes where category=\'{cats.iloc[7]['category']}\' order by random() limit {limit}
#     """
# ]

In [5]:
%%time
dfs = []

con = psycopg2.connect(dbname='mimic', user='sudarshan', host='/var/run/postgresql')
for q in queries:
    df = pd.read_sql_query(q, con)
    dfs.append(df)
con.close()
    
df = pd.concat(dfs)
df.reset_index(inplace=True, drop=True)
# df.set_index('row_id', inplace=True)
print(df.shape)

(1398, 2)
CPU times: user 35.4 ms, sys: 1.92 ms, total: 37.3 ms
Wall time: 3.05 s


In [ ]:
pat = re.compile(r'\[\*\*(.*?)\*\*\]', re.IGNORECASE)

In [ ]:
tpat = re.compile(r'\[\*\*(\d{2})\*\*\] \b[a|p].?m.?\b', re.IGNORECASE)

In [6]:
%%time
df['scrubbed'] = df['text'].apply(process_note)

CPU times: user 816 ms, sys: 0 ns, total: 816 ms
Wall time: 817 ms


In [7]:
test = df.iloc[964]['text']
# for m in pat.finditer(test):
#     print(m)

print(test)

CVICU
   HPI:
   [**6-9**] POD 2
   76M s/p CABG x4 (Lima->LAD/SVG->Diag/OM/PDA )/LAA resection-[**6-7**]
   EF:45-50% Cr:1.5 Wt:220 lbs H: 5'8'' HgA1c: 6.6
   PMH: MI 30y ago, remote tob, severe COPD, severe PVD, Afib HTN,
   ^lipids, Obesity, Glaucoma, Cholelithiasis, right pericardial Cyst
   (noted on CT [**5-12**]), skin CA x 2
   Chief complaint:
   PMHx:
   Current medications:
   Acetaminophen , Albuterol-Ipratropium , Albumin 5% (25g / 500mL) ,
   Amiodarone,  Aspirin , Calcium Gluconate , Calcium Chloride ,.
   Chlorhexidine Gluconate 0.12% Oral Rinse , Dextrose 50% , Docusate
   Sodium (Liquid) Fentanyl Citrate , Insulin , Latanoprost 0.005% Ophth.
   Soln. , Magnesium Sulfate , Metoclopramide , Milk of Magnesia ,
   Midazolam , Milrinone , Pantoprazole , Phenylephrine , Pneumococcal Vac
   Polyvalent , Potassium Chloride , Simvastatin , Sodium Chloride 0.9%
   Flush , Timolol Maleate 0.5% , Vasopressin
   24 Hour Events:
 TRANS ESOPHAGEAL ECHO - At [**2129-6-8**] 08:20 AM
 

In [8]:
out = process_note(test)
# for m in pat.finditer(out):
#     print(m)

print(out)

CVICU
   HPI:
   t_monthday POD 2
   76M s/p CABG x4 (Lima->LAD/SVG->Diag/OM/PDA )/LAA resection-t_monthday
   EF:45-50% Cr:1.5 Wt:220 lbs H: 5'8'' HgA1c: 6.6
   PMH: MI 30y ago, remote tob, severe COPD, severe PVD, Afib HTN,
   ^lipids, Obesity, Glaucoma, Cholelithiasis, right pericardial Cyst
   (noted on CT t_monthday), skin CA x 2
   Chief complaint:
   PMHx:
   Current medications:
   Acetaminophen , Albuterol-Ipratropium , Albumin 5% (25g / 500mL) ,
   Amiodarone,  Aspirin , Calcium Gluconate , Calcium Chloride ,.
   Chlorhexidine Gluconate 0.12% Oral Rinse , Dextrose 50% , Docusate
   Sodium (Liquid) Fentanyl Citrate , Insulin , Latanoprost 0.005% Ophth.
   Soln. , Magnesium Sulfate , Metoclopramide , Milk of Magnesia ,
   Midazolam , Milrinone , Pantoprazole , Phenylephrine , Pneumococcal Vac
   Polyvalent , Potassium Chloride , Simvastatin , Sodium Chloride 0.9%
   Flush , Timolol Maleate 0.5% , Vasopressin
   24 Hour Events:
 TRANS ESOPHAGEAL ECHO - At t_fulldate t_forenoon
 

In [ ]:
for i, row in df.iterrows():
    if len(tpat.findall(row['text'])) != 0:
        print(i, tpat.findall(row['text']))

In [ ]:
for i, row in df.iterrows():
    if len(tpat.findall(row['scrubbed'])) != 0:
        print(i, tpat.findall(row['scrubbed']))

In [ ]:
for i, row in df.iterrows():
    if len(pat.findall(row['scrubbed'])) != 0:
        print(i, pat.findall(row['scrubbed']))